# Práctica 1: Introducción a RDDs: tranformaciones y acciones Esta práctica es una introducción al uso de RDDs con spark. Trabajaremos con colecciones de datos sin esquema (RDD) y utilizaremos transformaciones y acciones para procesarlos. 

In [1]:
import math

# Cargamos el ficheros de alturas. Eliminamos los datos con '.' para pasar metros a centimetros y los datos como numéricos.
# También se filtran los datos para eliminar datos negativos, y como algunos datos al eliminar los '.' han podido quedar erroneos
# porque estaban por ejemplo como "1.8" en lugar de "1.80" se multiplican por 10 para que estén correctamente pasados a cms.
rdd_alt = sc.textFile("alturas_v0.csv").map(lambda a: a.replace('.','')).map(lambda a: int(a))
rdd_alt = rdd_alt.filter(lambda a: a>=0).map(lambda a: a*10 if len(str(a))<3 else a)

# se mapea una tupla y reduce para conseguir tener una tupla con el número de registros, la suma de ellos y la suma de los 
# productos. Con estos tres datos se puede calcular la media y desviación.
rdd_alt = rdd_alt.map(lambda a: (1,a,a*a)).reduce(lambda a, b: ((a[0]+b[0]),(a[1]+b[1]),(a[2]+b[2])))

print("La media es: ", rdd_alt[1]/rdd_alt[0])
print("La desviación típica es: ", math.sqrt((rdd_alt[2]/rdd_alt[0] - (rdd_alt[1]/rdd_alt[0])**2)))


La media es:  168.93825222396651
La desviación típica es:  7.8340557833708955


In [2]:
# Cargamos el ficheros de alturas. Eliminamos los datos con '.' para pasar metros a centimetros y los datos como numéricos.
# También se filtran los datos para eliminar datos negativos, y como algunos datos al eliminar los '.' han podido quedar erroneos
# porque estaban por ejemplo como "1.8" en lugar de "1.80" se multiplican por 10 para que estén correctamente pasados a cms.
# La diferencia a tener en cuenta es que en este fichero hay dos columnas, lo que implica uso de índices por ejemplo.
rdd_alt2 = sc.textFile("alturas.csv").map(lambda a: a.split(','))
rdd_alt2 = rdd_alt2.map(lambda a: (a[0],a[1].replace('.',''))).map(lambda a: (a[0],(int(a[1])*10)) if len(a[1])<3 else (a[0],a[1]))
rdd_alt2 = rdd_alt2.map(lambda a: (a[0],int(a[1]))).filter(lambda a: int(a[1])>=0)

rdd_alt2 = rdd_alt2.map(lambda a: (a[0],(1,a[1],a[1]*a[1]))).reduceByKey(lambda a, b: ((a[0]+b[0]),(a[1]+b[1]),(a[2]+b[2])))
rdd_alt2.cache()

# se ha obtenido un RDD con dos registros, uno para hombres y otro para mujeres con los datos requeridos en cada caso para 
# calcular la media y la desviación. Filtramos para obtener cada una de las tuplas de valores y realizar los cálculos.
rdd_h = rdd_alt2.filter(lambda a: a[0]=='H').map(lambda a: a[1]).reduce(lambda a: a)
rdd_m = rdd_alt2.filter(lambda a: a[0]=='M').map(lambda a: a[1]).reduce(lambda a: a)

print("La media de los hombres es: ", rdd_h[1]/rdd_h[0])
print("La media de las mujeres es: ", rdd_m[1]/rdd_m[0])
print("La desviación típica de los hombres es: ", math.sqrt((rdd_h[2]/rdd_h[0] - (rdd_h[1]/rdd_h[0])**2)))
print("La desviación típica de las mujeres es: ", math.sqrt((rdd_m[2]/rdd_m[0] - (rdd_m[1]/rdd_m[0])**2)))


La media de los hombres es:  173.46075433231397
La media de las mujeres es:  164.16774193548386
La desviación típica de los hombres es:  7.53122883663738
La desviación típica de las mujeres es:  4.684891325217995
